<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from query.datasets.prelude import *
from query.datasets.ingest import ingest_pose
import pyspark.sql.functions as func

In [ ]:
data = '''Lindsey Graham,U.S. Senate South Carolina,2003,Republican Party,
Richard Burr,U.S. Senate North Carolina,Republican Party,
Pat Toomey,U.S. Senate Pennsylvania,2011,Republican Party,
Lamar Alexander,U.S. Senate Tennessee,2003,Republican Party,
Shelley Moore Capito,U.S. Senate West Virginia,Republican Party,
Bob Corker,U.S. Senate Tennessee,2007,Republican Party,
Todd C. Young,U.S. Senate Indiana,Republican Party,
Pat Roberts,U.S. Senate Kansas,1997,Republican Party,
Ben Sasse,U.S. Senate Nebraska,Republican Party,
Bill Cassidy,U.S. Senate Louisiana,2015,Republican Party,
John Neely Kennedy,U.S. Senate Louisiana,Republican Party,
Thom Tillis,U.S. Senate North Carolina,Republican Party,
David Perdue,U.S. Senate Georgia,Republican Party,
Orrin Hatch,U.S. Senate Utah,Republican Party,
Daniel S. Sullivan,U.S. Senate Alaska,Republican Party,
Chuck Grassley,U.S. Senate Iowa,1981,Republican Party,
Steve Daines,U.S. Senate Montana,Republican Party,
Jim Inhofe,U.S. Senate Oklahoma,1995,Republican Party,
Jerry Moran,U.S. Senate Kansas,2011,Republican Party,
Rob Portman,U.S. Senate Ohio,2011,Republican Party,
Ron Johnson,U.S. Senate Wisconsin,2010,Republican Party,
Jeff Flake,U.S. Senate Arizona,2013,Republican Party,
Susan Collins,U.S. Senate Maine,Republican Party,
Cory Gardner,U.S. Senate Colorado,2015,Republican Party,
Tom Cotton,U.S. Senate Arkansas,Republican Party,
Dean Heller,U.S. Senate Nevada,Republican Party,
Ted Cruz,U.S. Senate Texas,Republican Party,
John Barrasso,U.S. Senate Wyoming,2007,Republican Party,
Mike Lee,U.S. Senate Utah,2011,Republican Party,
John Cornyn,U.S. Senate Texas,2003-01-03,Republican Party,
Deb Fischer,U.S. Senate Nebraska,Republican Party,
Roy Blunt,U.S. Senate Missouri,Republican Party,
Mitch McConnell,U.S. Senate Kentucky,1985,Republican Party,
John McCain,U.S. Senate Arizona,1987,Republican Party,
John Boozman,U.S. Senate Arkansas,2011,Republican Party,
Mike Enzi,U.S. Senate Wyoming,1997,Republican Party,
Mike Crapo,U.S. Senate Idaho,Republican Party,
Roger Wicker,U.S. Senate Mississippi,Republican Party,
Joni Ernst,U.S. Senate Iowa,Republican Party,
John Thune,U.S. Senate South Dakota,2005,Republican Party,
Jim Risch,U.S. Senate Idaho,Republican Party,
John Hoeven,U.S. Senate North Dakota,2011,Republican Party,
Mike Rounds,U.S. Senate South Dakota,Republican Party,
Johnny Isakson,U.S. Senate Georgia,Republican Party,
Marco Rubio,U.S. Senate Florida,Republican Party,
Richard Shelby,U.S. Senate Alabama,1987,Republican Party,
Lisa Murkowski,U.S. Senate Alaska,2003,Republican Party,
Rand Paul,U.S. Senate Kentucky,2011,Republican Party,
Tim Scott,U.S. Senate South Carolina,2013,Republican Party,
James Lankford,U.S. Senate Oklahoma,Republican Party,
Cindy Hyde-Smith,U.S. Senate Mississippi,2018-04-01,Republican Party,
Angus King,U.S. Senate Maine,2013,Independent,
Bernie Sanders,U.S. Senate Vermont,2007,Independent,
Jack Reed,U.S. Senate Rhode Island,1997,Democratic Party,
Chris Coons,U.S. Senate Delaware,Democratic Party,
Gary Peters,U.S. Senate Michigan,Democratic Party,
Maggie Hassan,U.S. Senate New Hampshire,Democratic Party,
Bob Menendez,U.S. Senate New Jersey,Democratic Party,
Tammy Duckworth,U.S. Senate Illinois,Democratic Party,
Dick Durbin,U.S. Senate Illinois,Democratic Party,
Mark Warner,U.S. Senate Virginia,Democratic Party,
Doug Jones,U.S. Senate Alabama,Democratic Party,
Brian E. Schatz,U.S. Senate Hawaii,2012,Democratic Party,
Chuck Schumer,U.S. Senate New York,Democratic Party,
Chris Van Hollen,U.S. Senate Maryland,Democratic Party,Chris Van Hollen
Michael Bennet,U.S. Senate Colorado,2009,Democratic Party,
Jeff Merkley,U.S. Senate Oregon,2009,Democratic Party,
Jeanne Shaheen,U.S. Senate New Hampshire,Democratic Party,
Ron Wyden,U.S. Senate Oregon,Democratic Party,
Tina Smith,U.S. Senate Minnesota,2018,Democratic Party,
Christopher S. Murphy,U.S. Senate Connecticut,Democratic Party,Chris Murphy
Tom Carper,U.S. Senate Delaware,Democratic Party,
Bill Nelson,U.S. Senate Florida,Democratic Party,
Mazie K. Hirono,U.S. Senate Hawaii,Democratic Party,
Joe Donnelly,U.S. Senate Indiana,2013-01-03,Democratic Party,
Tom Udall,U.S. Senate New Mexico,Democratic Party,
Debbie Stabenow,U.S. Senate Michigan,Democratic Party,
Amy Klobuchar,U.S. Senate Minnesota,Democratic Party,
Claire McCaskill,U.S. Senate Missouri,Democratic Party,
Martin Heinrich,U.S. Senate New Mexico,Democratic Party,
Kirsten Gillibrand,U.S. Senate New York,Democratic Party,
Heidi Heitkamp,U.S. Senate North Dakota,Democratic Party,
Bob Casey Jr.,U.S. Senate Pennsylvania,2007,Democratic Party,Bob Casey
Maria Cantwell,U.S. Senate Washington,2001,Democratic Party,
Tammy Baldwin,U.S. Senate Wisconsin,Democratic Party,
Ben Cardin,U.S. Senate Maryland,2007,Democratic Party,
Dianne Feinstein,U.S. Senate California,1992,Democratic Party,
Jon Tester,U.S. Senate Montana,Democratic Party,
Sherrod Brown,U.S. Senate Ohio,2007,Democratic Party,
Cory Booker,U.S. Senate New Jersey,Democratic Party,
Kamala D. Harris,U.S. Senate California,Democratic Party,
Catherine Cortez Masto,U.S. Senate Nevada,Democratic Party,
Elizabeth Warren,U.S. Senate Massachusetts,2013-01-03,Democratic Party,
Tim Kaine,U.S. Senate Virginia,Democratic Party,
Joe Manchin III,U.S. Senate West Virginia,Democratic Party,Joe Manchin
Patrick Leahy,U.S. Senate Vermont,Democratic Party,
Ed Markey,U.S. Senate Massachusetts,Democratic Party,
Richard Blumenthal,U.S. Senate Connecticut,Democratic Party,
Sheldon Whitehouse,U.S. Senate Rhode Island,2007,Democratic Party,
Patty Murray,U.S. Senate Washington,1993,Democratic Party,
'''

In [ ]:
def remove_middle_name(name):
    tokens = name.split(' ')
    if len(tokens) == 2:
        return tokens[0], tokens[1]
    elif len(tokens) == 3:
        if tokens[1] == 'Van':
            return tokens[0], tokens[1] + ' ' + tokens[2]
        else:
            return tokens[0], tokens[2]
    else:
        raise NotImplementedError('cannot handle {}'.format(name))

senators = []
senators_meta = {}
for line in data.split('\n'):
    line = line.strip()
    if line == '':
        continue
    tokens = line.split(',')
    if len(tokens) == 5:
        name, state, _, party, alt_name = tokens
    else:
        name, state, party, alt_name = tokens
    if alt_name:
        first, last = remove_middle_name(alt_name)
    else:
        first, last = remove_middle_name(name)
    senators.append((first, last))
    senators_meta[(first, last)] = (state, party)
senators.sort()

In [ ]:
senator_to_videos_ids = {}
for senator in senators:
    first, last = senator
    video_ids = set()
    result = caption_search([(first + ' ' + last).upper()])
    video_ids.update(result[0].keys())
    result = caption_search([('senator ' + last).upper()])
    video_ids.update(result[0].keys())
    senator_to_videos_ids[senator] = video_ids
    print(senator, len(video_ids))

In [ ]:
def get_count_for_show(show_name, video_ids):
    return Video.objects.filter(show__name=show_name, id__in=video_ids).count()

In [ ]:
shows = [
    'MSNBC Live',
    'CNN Newsroom Live',
    'Americas News HQ',
    'FOX  Friends',
    'The OReilly Factor',
    'Tucker Carlson Tonight',
    'The Rachel Maddow Show',
]
for senator in senators:
    print(senator)
    for show_name in shows:
        print('\t', show_name, get_count_for_show(show_name, senator_to_videos_ids[senator]))

In [ ]:
def plot_for_senator(senator, data):
    fig, ax1 = plt.subplots()
    ind = np.arange(len(data))
    width = 0.5
    rect1 = ax1.bar(ind - width / 2, [y for _, y, _ in data], width,
                    color='SkyBlue', label='Raw number of appearances')
    ax1.set_ylabel('Number of appearances', color='SkyBlue', fontsize=18)
    ax1.set_title('{}: Number of appearances by show'.format(' '.join(senator)))
    ax1.set_xticks(ind)
    ax1.set_xticklabels([x for x, _, _ in data])
    
    ax2 = ax1.twinx()
    rect2 = ax2.bar(ind + width / 2, [float(y)/z for _, y, z in data], width,
                   color='DarkRed', label='Normalized by total video count')
    ax2.set_ylabel('Normalized by total video count', color='DarkRed', fontsize=18)
    
    plt.show()
    
for senator in senators:
    print(senator, senators_meta[senator])
    data = []
    for show_name in shows:
        data.append((show_name, get_count_for_show(show_name, senator_to_videos_ids[senator]),
                     Video.objects.filter(show__name=show_name).count()))
    plot_for_senator(senator, data)